In [105]:
%%writefile task1.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.dynamic.partition=true;
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.error.on.empty.partition=true;

use akorzun_;

DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external(
id INT,
post_type_id INT,
creation_date DATE,
tags STRING,
owner_user_id INT,
parent_id INT,
score INT,
favorite_count INT
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
 "input.regex" = '^<row(?=.*\\bId="(\\d+)")(?=.*\\bPostTypeId="(\\d+)")(?=.*\\bCreationDate="(\\d{4}-\\d{2}-\\d{2})T\\d{2}:\\d{2}:\\d{2}[.]\\d{3}")(?=.*\\bTags="(&lt\;.+&gt\;)+")?(?=.*\\bOwnerUserId="(\\d+)")?(?=.*\\bParentId="(\\d+)")?(?=.*\\bScore="([-]?\\d+)")(?=.*\\bFavoriteCount="(\\d+)")?.*$' 
      )
LOCATION '/data/stackexchange1000/posts/';

Overwriting task1.hql


In [106]:
%%writefile -a task1.hql

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
id INT,
post_type_id INT,
tags STRING,
owner_user_id INT,
parent_id INT,
score INT,
favorite_count INT
)
PARTITIONED BY (year STRING, month STRING)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY  '\t';
--LOCATION '/user/jovyan/pdb/';

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT id, post_type_id, tags, owner_user_id, parent_id, score, favorite_count, substring(creation_date,0,4), substring(creation_date,0,7);

Appending to task1.hql


In [107]:
%%writefile -a task1.hql

SELECT sub.year,
       sub.month,
       sub.rcount
FROM
(SELECT year,
       month,
       COUNT(1) as rcount,
       ROW_NUMBER() OVER (ORDER BY month) as rownum
FROM posts_sample 
GROUP BY year, month) sub
WHERE sub.rownum = 3;

Appending to task1.hql


In [109]:
!hive -f task1.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.124 seconds
OK
Time taken: 1.713 seconds
OK
Time taken: 0.607 seconds
OK
Time taken: 3.315 seconds
OK
Time taken: 0.326 seconds
Query ID = jovyan_20180302172727_dd12c793-9560-4db5-a751-5a713e99bf86
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1519997433159_0039, Tracking URL = http://7e16be62487e:8088/proxy/application_1519997433159_0039/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1519997433159_0039
Hadoop job information for Stage-1: number of mappe

Partition akorzun_.posts_sample{year=2011, month=2011-02} stats: [numFiles=1, numRows=273, totalSize=12837, rawDataSize=12564]
Partition akorzun_.posts_sample{year=2011, month=2011-03} stats: [numFiles=1, numRows=349, totalSize=15866, rawDataSize=15517]
Partition akorzun_.posts_sample{year=2011, month=2011-04} stats: [numFiles=1, numRows=321, totalSize=14412, rawDataSize=14091]
Partition akorzun_.posts_sample{year=2011, month=2011-05} stats: [numFiles=1, numRows=327, totalSize=15406, rawDataSize=15079]
Partition akorzun_.posts_sample{year=2011, month=2011-06} stats: [numFiles=1, numRows=340, totalSize=15691, rawDataSize=15351]
Partition akorzun_.posts_sample{year=2011, month=2011-07} stats: [numFiles=1, numRows=335, totalSize=15551, rawDataSize=15216]
Partition akorzun_.posts_sample{year=2011, month=2011-08} stats: [numFiles=1, numRows=362, totalSize=16721, rawDataSize=16359]
Partition akorzun_.posts_sample{year=2011, month=2011-09} stats: [numFiles=1, numRows=343, totalSize=16296, raw

Partition akorzun_.posts_sample{year=2016, month=2016-12} stats: [numFiles=1, numRows=195, totalSize=10369, rawDataSize=10174]
Partition akorzun_.posts_sample{year=__HIVE_DEFAULT_PARTITION__, month=__HIVE_DEFAULT_PARTITION__} stats: [numFiles=1, numRows=384, totalSize=8064, rawDataSize=7680]
MapReduce Jobs Launched: 
Stage-Stage-1: Map: 1   Cumulative CPU: 46.97 sec   HDFS Read: 60004554 HDFS Write: 1969506 SUCCESS
Total MapReduce CPU Time Spent: 46 seconds 970 msec
OK
Time taken: 87.012 seconds
Query ID = jovyan_20180302172828_fb04c8e1-bfb6-44ed-8e24-f3267e78d5d2
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_151999743315